In [1]:
import pandas as pd
import numpy as np
from Merge import *
from sklearn.cross_validation import train_test_split
from model_builder import *


/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
train= pd.read_csv('balanced_data.csv')

test= pd.read_csv('imbalanced_data.csv') 
    
train['is_install'] = train['timestamp_at_install'].notnull() * 1
train.reset_index(drop=True, inplace=True)
test['is_install'] = test['timestamp_at_install'].notnull() * 1
test.reset_index(drop=True, inplace=True)


/Users/amiee/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
base_features =['publisher_app_store_id', 'advertiser_app_store_id','country_code']
#features_other_table= ['time_to_show_countdown']
#merge_id = ['creative_id']
yX_test = test[['is_install']+base_features].copy()
yX_train = train[['is_install']+base_features].copy()
#yX_train = yX_train[['is_install']+base_features+features_other_table].copy()


In [4]:
numerical_list =[]
string_list =['publisher_app_store_id','advertiser_app_store_id','country_code']
int_list =[]
timestamp_list =[]
#list_col = ['languages','genre_ids','genres','supported_devices','screenshot_urls','categories','device_type_whitelist','creative_tag']


In [5]:
a = preprocessing_data(yX_test,numerical_list,string_list,int_list,timestamp_list)
b = preprocessing_data(yX_train,numerical_list,string_list,int_list,timestamp_list)

In [6]:
##you can't put integer column into this
## if you add something, please change the preprocessing function as well.
def run_model(df,col):
    cont_features_pc = []
    cat_interactions_pc = []
    cat_features_pc =col
    other=[]
    y=['is_install']
    train, test = train_test_split(df, test_size=0.2)
    X_train, y_train, X_test, y_test = train, train.pop('is_install'), test, test.pop('is_install')
    # Feature Hashing

    feature_creator = FeatureCreator()
    design_matrix_transformer = FeatureHasher(18, cat_features_pc, None, cat_interactions_pc, store_fmap=True) # You can experiment with hasher bits (we used 18 here)

    X_test = feature_creator.transform(X_test, inplace=True)
    X_test, f_map = design_matrix_transformer.fit_transform(X_test)

    X_train = feature_creator.transform(X_train, inplace=True)
    X_train, f_map = design_matrix_transformer.fit_transform(X_train)

    logistic_baseline = logistic_model(X_train, y_train)

    # Calculate prediction/probability of train and test
    X_train_predictions = logistic_baseline.predict(X_train)
    X_train_predprob = logistic_baseline.predict_proba(X_train)[:, 1]

    X_test_predictions = logistic_baseline.predict(X_test)
    X_test_predprob = logistic_baseline.predict_proba(X_test)[:, 1]

    # Calculate metrics of train, validation and test set.
    lr_ll_val = -logistic_baseline.best_score_

    lr_ll_train = log_loss(y_train, X_train_predprob)
    lr_auc_train = roc_auc_score(y_train, X_train_predprob)

    lr_ll_test = log_loss(y_test, X_test_predprob)    
    lr_auc_test = roc_auc_score(y_test, X_test_predprob)

    #Print out the results
    print "Best parameter: ", logistic_baseline.best_params_

    print "Log Loss (Validation): %f" % lr_ll_val

    print "Log Loss (Train): %f" % lr_ll_train
    print "AUC (Train): %f" % lr_auc_train
 
    print 'Log Loss (Test): %f' % lr_ll_test
    print 'AUC (Test): %f' % lr_auc_test
    return lr_ll_train,lr_ll_test,lr_auc_train,lr_auc_test



In [7]:
result = pd.DataFrame(columns=['train_loss','test_loss','train_auc','test_auc'],index = ['imbalanced_train_on_imbalanced_test','baseline_imbalanced','balanced_train_onimbalanced_test','baseline_balanced'])

In [8]:
b.head()

,is_install,publisher_app_store_id,advertiser_app_store_id,country_code
0,0,5255b9a777aa478519000008,1009442510,CN
1,0,507686ae771615941001aca5,5743f032a5a36ff4300000a5,ID
2,0,52a861166faa22fa48000009,727296976,CA
3,0,5552167b1f1ac071400000d9,57a28ffbb019f8257c00021d,LA
4,0,5759694c1c7b92224b000039,com.plarium.vikings,CZ


In [9]:
i ='baseline_imbalanced'
col = base_features
result.loc[i,'train_loss'],result.loc[i,'test_loss'],result.loc[i,'train_auc'],result.loc[i,'test_auc']=run_model(a,col)

Train Regression Model


/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127

Best parameter:  {'C': 0.6}
Log Loss (Validation): 0.419789
Log Loss (Train): 0.410968
AUC (Train): 0.776260
Log Loss (Test): 0.419506
AUC (Test): 0.765367


In [10]:
##you can't put integer column into this
## if you add something, please change the preprocessing function as well.
def run_model_balanced(train,test,col):
    cont_features_pc = []
    cat_interactions_pc = []
    cat_features_pc =col
    other=[]
    y=['is_install']

    X_train, y_train, X_test, y_test = train.iloc[:,1:], train.iloc[:,0], test.iloc[:,1:], test.iloc[:,0]
    # Feature Hashing

    feature_creator = FeatureCreator()
    design_matrix_transformer = FeatureHasher(18, cat_features_pc, None, cat_interactions_pc, store_fmap=True) # You can experiment with hasher bits (we used 18 here)

    X_test = feature_creator.transform(X_test, inplace=True)
    X_test, f_map = design_matrix_transformer.fit_transform(X_test)

    X_train = feature_creator.transform(X_train, inplace=True)
    X_train, f_map = design_matrix_transformer.fit_transform(X_train)

    logistic_baseline = logistic_model(X_train, y_train)

    # Calculate prediction/probability of train and test
    X_train_predictions = logistic_baseline.predict(X_train)
    X_train_predprob = logistic_baseline.predict_proba(X_train)[:, 1]

    X_test_predictions = logistic_baseline.predict(X_test)
    X_test_predprob = logistic_baseline.predict_proba(X_test)[:, 1]

    # Calculate metrics of train, validation and test set.
    lr_ll_val = -logistic_baseline.best_score_

    lr_ll_train = log_loss(y_train, X_train_predprob)
    lr_auc_train = roc_auc_score(y_train, X_train_predprob)

    lr_ll_test = log_loss(y_test, X_test_predprob)    
    lr_auc_test = roc_auc_score(y_test, X_test_predprob)

    #Print out the results
    print "Best parameter: ", logistic_baseline.best_params_

    print "Log Loss (Validation): %f" % lr_ll_val

    print "Log Loss (Train): %f" % lr_ll_train
    print "AUC (Train): %f" % lr_auc_train
 
    print 'Log Loss (Test): %f' % lr_ll_test
    print 'AUC (Test): %f' % lr_auc_test
    return lr_ll_train,lr_ll_test,lr_auc_train,lr_auc_test



In [11]:
i ='baseline_balanced'
col = base_features
result.loc[i,'train_loss'],result.loc[i,'test_loss'],result.loc[i,'train_auc'],result.loc[i,'test_auc']=run_model_balanced(b,a,col)

Train Regression Model


/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/Users/amiee/anaconda2/lib/python2.7/site-packages/sklearn/metrics/scorer.py:127

Best parameter:  {'C': 0.6}
Log Loss (Validation): 0.578392
Log Loss (Train): 0.569588
AUC (Train): 0.774322
Log Loss (Test): 0.578436
AUC (Test): 0.767144
